##import the libs

In [ ]:
import os
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from mpl_toolkits.axes_grid1 import ImageGrid

import cv2
from glob import glob

import tensorflow as tf
from tensorflow.keras import Input
from tensorflow.keras.models import Model, load_model, save_model
from tensorflow.keras.layers import Input, Activation, BatchNormalization, Dropout, Lambda, Conv2D
from tensorflow.keras.layers import Conv2DTranspose, MaxPooling2D, concatenate, AveragePooling2D, Dense, Flatten, MaxPool2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping, ModelCheckpoint, ReduceLROnPlateau

from tensorflow.keras import backend as K
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
%cd /content/drive/MyDrive/Colab Notebooks/

In [ ]:
#!unzip dataset.zip

In [ ]:
train_dir_img = 'dataset/3_channel/' 
train_dir_mask = 'dataset/mask/'

In [ ]:
img_h = 256
img_w = 256
batch_size = 4


In [ ]:
train_imgs = []
for path, subdirs, files in os.walk(train_dir_img):
    for name in files:
        train_imgs.append(os.path.join(path, name))

In [ ]:
train_imgs

In [ ]:
len(train_imgs)

In [ ]:
train_masks = []
for path, subdirs, files in os.walk(train_dir_mask):
    for name in files:
        train_masks.append(os.path.join(path, name))

In [ ]:
train_masks

In [ ]:
len(train_masks)

In [ ]:
train_imgs.sort()
train_masks.sort()

In [ ]:
train_imgs

In [ ]:
train_masks

In [ ]:
# Define X and Y as number of images along with shape of one image
X_t = np.zeros((len(train_imgs),img_h,img_w,3), dtype=np.float32)
y_t = np.zeros((len(train_imgs),img_h,img_w,3), dtype=np.float32)

In [ ]:
print(X_t.shape)

print(y_t.shape)

In [ ]:
for i, i_d in enumerate(train_imgs):
    
    img = cv2.imread(i_d)
    img = cv2.resize(img, (img_h, img_w))
    X_t[i] = img

In [ ]:
X_t

In [ ]:
X_t.shape

In [ ]:
for i, i_d in enumerate(train_masks):
    img = cv2.imread(i_d)
    #print(img)
    img = cv2.resize(img, (img_h, img_w))
    y_t[i] = img

In [ ]:
y_t

In [ ]:
y_t.shape

In [ ]:
X_t[0].shape

In [ ]:
y_t[0]

In [ ]:
# from keras.preprocessing.image import ImageDataGenerator

# train_gen = ImageDataGenerator(rescale = 1./255)
#   # rotation_range=30,
#   #                             height_shift_range=0.3,
#   #                             zoom_range=0.3,
#   #                             horizontal_flip=True,
#   #                             width_shift_range=0.3,
#   #                             fill_mode='reflect
# val_gen = ImageDataGenerator(rescale=1./255)

In [ ]:
from sklearn.model_selection import train_test_split

xt, xv, yt, yv = train_test_split(X_t, y_t, test_size=0.2)

In [ ]:
from skimage.io import imshow
y = yv[22][:,:,0]
imshow(y)
plt.show()

In [ ]:
y = xv[22][:,:,0]
imshow(y)
plt.show()

In [ ]:
# seed=42
# def combineGenerator(gen1, gen2):
#     while True:
#         yield(gen1.next(), gen2.next())

# train_generator_image = train_gen.flow(xt,batch_size=16, seed=seed, shuffle=True)
# train_generator_mask = train_gen.flow(yt, batch_size=16, seed=seed, shuffle=True)
# train_generator = combineGenerator(train_generator_image, train_generator_mask)
# validation_generator_image = val_gen.flow(xv, batch_size=16, seed=seed, shuffle=True)
# validation_generator_mask = val_gen.flow(yv, batch_size=16, seed=seed, shuffle=True)
# validation_generator = combineGenerator(validation_generator_image, validation_generator_mask)

In [ ]:
# a, b = next(validation_generator)
# print(a.shape, b.shape)

In [ ]:
# img1, mask1 = next(train_generator)

# fig=plt.figure(figsize=(12,12))
# columns = 2
# rows = 4
# counter = 0
# c = 0
# for i in range(1, columns*rows +1):
#     #img = np.random.randint(10, size=(h,w))
#     fig.add_subplot(rows, columns, i)
#     if counter%2 == 0:
#         plt.imshow(img1[c])
#     else:
#         plt.imshow(mask1[c][:,:,0])
#         c = c + 1
#     counter = counter + 1
    
# plt.show()

In [ ]:
def bn_act(x, act=True):
    x = keras.layers.BatchNormalization()(x)
    if act==True:
        x = keras.layers.Activation('relu')(x)
    return x

def conv_block(x, filters, kernel_size=(3,3), padding='same', strides=1):
    conv = bn_act(x)
    conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(conv)
    return conv

def stem(x, filters, kernel_size=(3,3), padding='same', strides=1):
    conv = keras.layers.Conv2D(filters, kernel_size, padding=padding, strides=strides)(x)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    
    shortcut = keras.layers.Conv2D(filters, kernel_size=(1,1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = keras.layers.Add()([conv, shortcut])
    return output

def residual_block(x, filters, kernel_size=(3,3), padding='same', strides=1):
    conv = conv_block(x, filters, kernel_size=kernel_size, padding=padding, strides=strides)
    conv = conv_block(conv, filters, kernel_size=kernel_size, padding=padding, strides=1)
    
    shortcut = keras.layers.Conv2D(filters, kernel_size=(1,1), padding=padding, strides=strides)(x)
    shortcut = bn_act(shortcut, act=False)
    
    output = keras.layers.Add()([shortcut, conv])
    return output

def up_sampling_concat_block(x, x_skip):
    up_sample = keras.layers.UpSampling2D((2,2))(x)
    concat = keras.layers.Concatenate()([up_sample, x_skip])
    return concat
    

In [ ]:
def ResUnet():
    f = [16,32,64,128,256]
    img_=256
    inputs = keras.layers.Input((img_, img_,3))
    
    #encoding
    e0 = inputs
    e1 = stem(e0, f[0])
    e2 = residual_block(e1, f[1], strides=2)
    e3 = residual_block(e2, f[2], strides=2)
    e4 = residual_block(e3, f[3], strides=2)
    e5 = residual_block(e4, f[4], strides=2)
    
    #bridge
    b0 = conv_block(e5, f[4], strides=1)
    b1 = conv_block(b0, f[4], strides=1)
    
    #decoder
    u0 = up_sampling_concat_block(b1, e4)
    d0 = residual_block(u0, f[4])
    
    u1 = up_sampling_concat_block(d0, e3)
    d1 = residual_block(u1, f[3])
    
    u2 = up_sampling_concat_block(d1, e2)
    d2 = residual_block(u2, f[2])
    
    u3 = up_sampling_concat_block(d2, e1)
    d3 = residual_block(u3, f[1])
    
    output = keras.layers.Conv2D(3, (1,1), padding='same', activation='sigmoid')(d3)
    model = keras.models.Model(inputs, output)
    return model

In [ ]:
from tensorflow.keras.layers import Flatten

In [ ]:
smooth = 1.

def dice_coef(y_true, y_pred):
    flatten_layer = Flatten()
    y_true_f = flatten_layer(y_true)
    y_pred_f = flatten_layer(y_pred)
    intersection = tf.reduce_sum(y_true_f * y_pred_f)
    return (2. * intersection + smooth) / (tf.reduce_sum(y_true_f) + tf.reduce_sum(y_pred_f) + smooth)


def dice_coef_loss(y_true, y_pred):
    return 1.0 - dice_coef(y_true, y_pred)

In [ ]:
from tensorflow.keras.optimizers import Adam
from tensorflow import keras
from tensorflow.keras import layers
model = ResUnet()
model.compile(optimizer=Adam(lr=0.0005), loss=dice_coef_loss, metrics=[dice_coef])
model.summary()

In [ ]:
from tensorflow.keras.callbacks import  ModelCheckpoint

callbacks=[ModelCheckpoint(monitor='val_loss',
                             filepath='dsb.h5',
                             save_best_only=True)]

history = model.fit(xt,yt, epochs = 20,
                              validation_data = (xv,yv), callbacks=callbacks)

In [ ]:
history.history.keys()

In [ ]:
plt.figure(figsize=(8,15))
plt.subplot(3,1,1)
plt.plot(model.history.history['loss'], 'b-', label='train_loss')
plt.plot(model.history.history['val_loss'], 'r-', label='val_loss')
plt.legend(loc='best')
plt.title('Loss')

# plt.subplot(3,1,2)
# plt.plot(model.history.history['dice_coef'], 'b-', label='train_iou')
# plt.plot(model.history.history['val_dice_coef'], 'r-', label='val_iou')
# plt.legend(loc='best')
# plt.title('IoU')

plt.subplot(3,1,2)
plt.plot(model.history.history['dice_coef'], 'b-', label='train_dice_coef')
plt.plot(model.history.history['val_dice_coef'], 'r-', label='val_dice_coef')
plt.legend(loc='best')
plt.title('Dice Coef')

In [ ]:
model.evaluate(xv, yv)


In [ ]:
# Results of Validation Dataset
def VisualizeResults(index):
    img = xv[index]
    img = img[np.newaxis, ...]
    pred_y = model.predict(img)
    pred_mask = tf.argmax(pred_y[0], axis=-1)
    pred_mask = pred_mask[..., tf.newaxis]
    fig, arr = plt.subplots(1, 3, figsize=(15, 15))
    arr[0].imshow(xv[index,:,:,0])
    arr[0].set_title('Processed Image')
    arr[1].imshow(yv[index,:,:,0])
    arr[1].set_title('Actual Masked Image ')
    arr[2].imshow(pred_mask[:,:,0])
    arr[2].set_title('Predicted Masked Image ')

In [ ]:
index = 35
VisualizeResults(index)

In [ ]:
model = keras.models.load_model('dsb.h5', custom_objects={'dice_coef_loss': dice_coef_loss,'dice_coef':dice_coef})


In [ ]:
test_mask = model.predict(xv, verbose=1)

In [ ]:
test_mask[0].shape

In [ ]:
#preds_test_t = (test_mask > 0.5)

In [ ]:
r = random.randint(0, len(xv)-1)

fig = plt.figure(figsize = (10,10))
fig.subplots_adjust(hspace=0.2, wspace=0.2)
ax = fig.add_subplot(1, 3, 1)
ax.imshow(xv[r][:,:,0])
ax = fig.add_subplot(1, 3, 2)
ax.imshow(yv[r][:,:,0])

ax = fig.add_subplot(1,3,3)
ax.imshow(test_mask[r][:,:,0])
